In [185]:
import json
import logging
import os

from bs4 import BeautifulSoup
import requests
import ricecooker

from engageny_chef import get_text, get_parsed_html_from_url, make_fully_qualified_url
from engageny_chef import ENGAGENY_CC_START_URL
from engageny_chef import DATA_DIR, TREES_DATA_DIR, CRAWLING_STAGE_OUTPUT
from engageny_chef import LOGGER
LOGGER.addHandler(logging.StreamHandler())  # needed for logging in to work in notebook

from pprint import pprint as pp
from re import compile

In [181]:
# basic reconaissance ... 
doc = get_parsed_html_from_url(ENGAGENY_CC_START_URL)

dual_toc_div = doc.find('div', id='mini-panel-common_core_curriculum')
ELA_toc = dual_toc_div.find('div', class_='panel-col-first')
MATH_toc = dual_toc_div.find('div', class_='panel-col-last')
MATH_grades_lis = MATH_toc.find_all('li')
ELA_grades_lis = ELA_toc.find_all('li')
MATH_grades = []

In [182]:
for grade_li in MATH_grades_lis:
    grade_path = grade_li.find('a')['href']
    grade_url = make_fully_qualified_url(grade_path)
    MATH_grades.append({
        'type': 'grade',
        'url': grade_url,
        'title': get_text(grade_li)
    })

In [183]:
pp(MATH_grades)

[{'title': 'Prekindergarten Mathematics',
  'type': 'grade',
  'url': 'https://www.engageny.org/content/prekindergarten-mathematics'},
 {'title': 'Kindergarten Mathematics',
  'type': 'grade',
  'url': 'https://www.engageny.org/resource/kindergarten-mathematics'},
 {'title': 'Grade 1 Mathematics',
  'type': 'grade',
  'url': 'https://www.engageny.org/resource/grade-1-mathematics'},
 {'title': 'Grade 2 Mathematics',
  'type': 'grade',
  'url': 'https://www.engageny.org/resource/grade-2-mathematics'},
 {'title': 'Grade 3 Mathematics',
  'type': 'grade',
  'url': 'https://www.engageny.org/resource/grade-3-mathematics'},
 {'title': 'Grade 4 Mathematics',
  'type': 'grade',
  'url': 'https://www.engageny.org/resource/grade-4-mathematics'},
 {'title': 'Grade 5 Mathematics',
  'type': 'grade',
  'url': 'https://www.engageny.org/resource/grade-5-mathematics'},
 {'title': 'Grade 6 Mathematics',
  'type': 'grade',
  'url': 'https://www.engageny.org/resource/grade-6-mathematics'},
 {'title': 'Gra

In [189]:
grade_page = get_parsed_html_from_url(MATH_grades[0]['url'])

In [190]:
pp(grade_page)

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML+RDFa 1.0//EN"
  "http://www.w3.org/MarkUp/DTD/xhtml-rdfa-1.dtd">

<html dir="ltr" lang="en">
<head profile="http://www.w3.org/1999/xhtml/vocab">
<meta charset="utf-8">
<meta content="width=device-width, initial-scale=1.0" name="viewport">
<!--[if IE]><![endif]-->
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="1505216248" property="og:updated_time"/>
<link href="/resource/prekindergarten-mathematics-module-1" rel="next"/>
<meta content="Drupal 7 (http://drupal.org)" name="Generator"/>
<link href="https://www.engageny.org/sites/all/themes/eny_subtheme/favicon.ico" rel="shortcut icon" type="image/vnd.microsoft.icon"/>
<title>Prekindergarten Mathematics | EngageNY</title>
<link href="https://www.engageny.org/sites/default/files/advagg_css/css__Tiwjk7T82ltEb-tTLH6R74C4VJO4igPPpgxT0qkSQ3k__ECnd9KWt6ldARrVmM2_WOMRvkrTXWBWzFg0SMFL8BKE__6N-1OvjeKrhGgvGIL7nYrG1jrrNOAz02CZkDuqPWmlQ.css" media="all" rel="stylesheet" type

In [191]:
pp(grade_curriculum_toc)

<div class="nysed-book-outline curriculum-map"><div class="tree" id="nid-26481"><ul class="list-unstyled"> <li class="collection-row has-list-items module depth-0">
<div class="details">
<a class="map-drop " href="#"><span class="drop-text fa" role="button"></span><span class="sr-only">Toggle Module 1</span></a>
<a class="item-link" href="/resource/grade-1-mathematics-module-1" title="Grade 1 Mathematics Module 1">
<strong class="item-title">Module 1</strong>
<p class="description"></p>
</a>
</div>
<div class="tree" id="nid-4591"><ul class="list-unstyled"> <li class="collection-row has-list-items topic depth-1">
<div class="details">
<a class="map-drop " href="#"><span class="drop-text fa" role="button"></span><span class="sr-only">Toggle Topic A</span></a>
<a class="item-link" href="/resource/grade-1-mathematics-module-1-topic" title="Grade 1 Mathematics Module 1, Topic A">
<strong class="item-title">Topic A</strong>
<p class="description"></p>
</a>
</div>
<div class="tree" id="nid-25

In [212]:
grade_url_re = compile(r'^/resource/(\w)+-(\w)+-module-(\d)+$')

In [213]:
pp(grade_url_re2.match('/resource/prekindergarten-mathematics-module-1'))

<_sre.SRE_Match object; span=(0, 46), match='/resource/prekindergarten-mathematics-module-1'>


In [214]:
for grade in MATH_grades:
    grade_page = get_parsed_html_from_url(grade['url'])
    grade_curriculum_toc = grade_page.find('div', class_='nysed-book-outline curriculum-map')
    for module_li in grade_curriculum_toc.find_all('li', class_='module'):
        details = module_li.find('div', class_='details').find('a', attrs={'href': grade_url_re })
        topics = []
        for topic in module_li.find('div', class_='tree').find_all('li', class_='topic'):
            pass
        grade_children.append({
            'title': get_text(details),
            'url': make_fully_qualified_url(details['href']),
            'topics': topics,
        })

TypeError: 'NoneType' object is not subscriptable

In [4]:
## Debugging web_resource_tree.json

In [5]:
def print_web_resource_tree(web_resource_tree):
    print('----')
    print('WEB RESOURCE TREE:', 'title:', web_resource_tree['title'], '  len(children) =', len(web_resource_tree['children']))
    # print('      description:', web_resource_tree['description'][0:60]+'..')
    for category in web_resource_tree['children']:
        print('   - Category   title:', category['title']) # len(category['title']))
        # print('           desciption:', category['description'][0:60]+'..') # len(category['description']))

        for resource in category['children']:
            # print(resource)
            if 'kind' not in resource:
                resource['kind'] = resource['type']
            print('      - Resource (%s):' % resource['kind'], resource['title'])
            for child in resource['children']:
                # print(child)
                print('         - Child (%s):' % child['kind'], child['title'])
    print('\n\n')

    

with open(os.path.join(TREES_DATA_DIR,CRAWLING_STAGE_OUTPUT)) as json_file:
    web_resource_tree = json.load(json_file)
    print_web_resource_tree(web_resource_tree)



----
WEB RESOURCE TREE: title: Engage NY Web Resource Tree (ELS and CCSSM)   len(children) = 0





## Calling chef method for debugging...


In [4]:
from engageny_chef import EngageNYChef
chef = EngageNYChef()

chef_args = None
chef_options = {}
chef.crawl(chef_args, chef_options)


Crawling results stored in chefdata/trees/web_resource_tree.json
